In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

/home/yutanagano/Projects/sceptr_publication_analyses/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokeniser = BertTokenizer.from_pretrained("wukevin/tcr-bert")
model = BertModel.from_pretrained("wukevin/tcr-bert")

In [3]:
df = pd.read_csv("../tcr_data/preprocessed/benchmarking/vdjdb_cleaned.csv")

In [4]:
df.head()

,TRAV,CDR3A,TRAJ,TRBV,CDR3B,TRBJ,Epitope,MHCA,MHCB,Epitope_species,Reference
0,TRAV27*01,CAGAGSQGNLIF,TRAJ42*01,TRBV19*01,CASSSRSSYEQYF,TRBJ2-7*01,GILGFVFTL,HLA-A*02:01:48,B2M,InfluenzaA,PMID:12796775
1,TRAV27*01,CAGAGSQGNLIF,TRAJ42*01,TRBV19*01,CASSSRSSYEQYF,TRBJ2-7*01,GILGFVFTL,HLA-A*02:01:48,B2M,InfluenzaA,PMID:18275829
2,TRAV27*01,CAGAGSQGNLIF,TRAJ42*01,TRBV19*01,CASSSRASYEQYF,TRBJ2-7*01,GILGFVFTL,HLA-A*02:01:48,B2M,InfluenzaA,PMID:18275829
3,TRAV24*01,CARNTGNQFYF,TRAJ49*01,TRBV6-5*01,CASSPVTGGIYGYTF,TRBJ1-2*01,NLVPMVATV,HLA-A*02:01:98,B2M,CMV,PMID:19542454
4,TRAV5*01,CAEDNNARLMF,TRAJ31*01,TRBV20-1*01,CSARDGTGNGYTF,TRBJ1-2*01,GLCTLVAML,HLA-A*02:01:48,B2M,EBV,PMID:21124993


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model = model.to(device)

In [6]:
def separate_tokens(cdr3: str) -> str:
    return " ".join(list(cdr3))

alpha_cdr3s = df.CDR3A.map(separate_tokens).to_list()
beta_cdr3s = df.CDR3B.map(separate_tokens).to_list()

In [7]:
tokens = tokeniser(beta_cdr3s, return_tensors="pt", padding=True)
tokens_on_device = {k: v.to(device) for k, v in tokens.items()}

In [10]:
with torch.no_grad():
    output = model(**tokens_on_device)

In [16]:
type(output.pooler_output)

torch.Tensor

In [ ]:
torch.